In [1]:
import os.path
import re
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import json
from collections import OrderedDict
import sys
import bs4
import json
import re

In [4]:
with open('Untitled', encoding = 'utf-8') as json_file:
    list_before = json.load(json_file)

list_movie = list_before['movieListResult']['movieList']
len_movie = len(list_movie)

In [301]:
class watcha_crawler(webdriver.Chrome):
    
    def log_in(self, id, password):
        self.get('https://watcha.com/ko-KR')
        now_html = self.page_source
        now_source = BeautifulSoup(now_html, 'lxml')

        #로그인 클릭 하는 코드
        self.find_elements_by_tag_name("li")[1].click()

        now_html = self.page_source
        now_source = BeautifulSoup(now_html, 'lxml')
        # 로그인 정보 넘겨주는 코드
        self.find_element_by_id("sign_in_email").send_keys(id)
        self.find_element_by_id("sign_in_password").send_keys(password)
        self.find_elements_by_tag_name("button")[6].click()
        
    
    def find_matching(self, list_movie, i): # 영화 리스트와 순서를 인풋으로 받으면
        
        count = 0
        name_movie_pre = list_movie[i]['movieNm'] # 영화 리스트에서 매칭 쿼리를 잡고
        name_movie = re.sub(' ','%20',name_movie_pre)
        year_movie = list_movie[i]['prdtYear'] 
        nation_movie = list_movie[i]['repNationNm'] 
        self.get('https://watcha.com/ko-KR/search?query='+name_movie) # 왓챠에서 영화를 검색해 본다.
        now_html = self.page_source
        now_source = BeautifulSoup(now_html, 'lxml')
        title_prev = now_source.find_all('li', {'class':"css-106b4k6-Self e3fgkal0" }) # 검색 결과 다양한 영화가 나오니 listup을 한다.
        len_same = len(title_prev)
        
        print(name_movie_pre + '에 대한 결과')
        print('정답은 {}, {}, {}'.format(name_movie_pre, year_movie, nation_movie))

        for j in range(len_same): #제목, 제작년도, 국가를 매칭시켜서 맞는 걸 찾는다.
            query_title = title_prev[j].a.find_all('div')[2].find_all('div')[0].text
            query_movie = title_prev[j].a.find_all('div')[2].find_all('div')[1].text.split()[0]
            query_nation = title_prev[j].a.find_all('div')[2].find_all('div')[1].text.split()[2]
            
            if(query_title == name_movie_pre and query_movie == year_movie and query_nation == nation_movie): # 세 조건이 맞으면
                print(name_movie_pre + '매칭 성공\n##########################################\n')
                movie_url = 'https://watcha.com' + title_prev[j].a['href'] # 매칭되는 영화 페이지를 리턴
                count+=1
                return movie_url
        
        if(count<1):
            if(len_same<1):
                print('검색 결과가 없음\n##########################################\n')
            else:
                print('검색 결과는 있지만 매칭되는 게 없음\n##########################################\n')
            movie_url = 0
            return movie_url

    def save_synopsis(self, movie_url):
        synopsis = ''
        if(movie_url != 0): # 매칭되는 영화가 있었으면
            self.get(movie_url)
            now_html = self.page_source
            now_source = BeautifulSoup(now_html, 'lxml')
            synopsis_prev = now_source.find_all('div', {'class':"css-1jyvmaq-ViewMore et86el20"})[0]
            synopsis_prev2 = synopsis_prev.a['href']
            self.get('https://watcha.com' + synopsis_prev2)
            now_html = self.page_source
            now_source = BeautifulSoup(now_html, 'lxml')
            synopsis = now_source.find_all('dd', {'class':"css-77qx4t-SummaryDetail e1kvv3954"})[0].text
    
        return synopsis
    
    def save_comments(self, movie_url, num_comments):
        if(movie_url != 0): # 매칭되는 영화가 있었으면
            self.get(movie_url+'/comments')
            for i in range(num_comments//3): #스크롤 내리는 코드. 로그인 해야함. num_comments만큼 댓글 수집. 
                crawler.execute_script("window.scrollTo(0, document.body.scrollHeight);") #한 번 스크롤에 3개씩 추가. 검증은 해봐야.
            now_html = crawler.page_source
            now_source = BeautifulSoup(now_html, 'lxml')
            comments_list = now_source.find_all('div', {'class':"css-wnwcvo-Comment e1oskw6f0"}) # 댓글 목록 수집
            how_many = len(comments_list)

            members = []
            
            for i in range(how_many):
                one_member = {}
                # 스포일러가 있는 경우와 없는 경우 나눠서 크롤링
                if str(type(comments_list[i].find('div', {'class': "css-1l07b40-Text-handleRenderInner e1xxz10x0"}))) != "<class 'NoneType'>":
                    one_member['comment'] = (comments_list[i].find('div', {'class': "css-1l07b40-Text-handleRenderInner e1xxz10x0"})).text
                else:
                    one_member['comment'] = comments_list[i].find('div', {'class': "css-v0bib7-Text-handleRenderInner e1xxz10x0"}).text
                one_member['ratings'] = comments_list[i].find_all('span')[0].text
                one_member['like'] = comments_list[i].find_all('em')[0].text
                one_member['dont_like'] = comments_list[i].find_all('em')[1].text
                members.append(one_member)
            
            return members
    
    

In [302]:
wd = "./chromedriver"
crawler = watcha_crawler(wd)

In [303]:
crawler.log_in('kpdpkp@naver.com','meanimo123') # 댓글 더보기를 하려면 로그인 해야 함.

synopsis = []
comments = []

In [319]:
all_data = {}

In [320]:
for i in range(len_movie):
    data_movie = {}
    data_movie['movieName'] = list_movie[i]['movieNm']
    movie_url = crawler.find_matching(list_movie,i)
    data_movie['synopsis'] = crawler.save_synopsis(movie_url)
    data_movie['comments'] = crawler.save_comments(movie_url,100)
    all_data[i] = data_movie

성우는 괜찮아에 대한 결과
정답은 성우는 괜찮아, 2017, 한국
검색 결과가 없음
##########################################

뷰티풀 보이에 대한 결과
정답은 뷰티풀 보이, 2018, 미국
뷰티풀 보이매칭 성공
##########################################

더 몬스터에 대한 결과
정답은 더 몬스터, 2016, 미국
더 몬스터매칭 성공
##########################################

침입자들에 대한 결과
정답은 침입자들, 2019, 홍콩
침입자들매칭 성공
##########################################

극장판 타오르지마버스터-블랙어썰트의 귀환에 대한 결과
정답은 극장판 타오르지마버스터-블랙어썰트의 귀환, 2019, 한국
검색 결과가 없음
##########################################

아마추어 유부녀에 대한 결과
정답은 아마추어 유부녀, 2019, 일본
검색 결과가 없음
##########################################

거근에 중독된 미모의 내 아내에 대한 결과
정답은 거근에 중독된 미모의 내 아내, 2019, 일본
검색 결과가 없음
##########################################

애월에 대한 결과
정답은 애월, 2019, 한국
애월매칭 성공
##########################################

갈까부다에 대한 결과
정답은 갈까부다, 2018, 한국
갈까부다매칭 성공
##########################################

섬에 대한 결과
정답은 섬, 2016, 한국
검색 결과는 있지만 매칭되는 게 없음
##########################################



In [327]:
all_data

{0: {'movieName': '성우는 괜찮아', 'synopsis': '', 'comments': None},
 1: {'movieName': '뷰티풀 보이',
  'synopsis': '내 삶의 빛, 나의 모든 것\r\n나의 아름다운 소년아, \r\n눈부신 그때로 돌아와줘\r\n \r\n“아름다운 소년이 있었어. 불행히도 소년은 끔찍한 병에 걸렸어”\r\n열렬한 독서가이자 재능 있는 예술가, 운동을 좋아하는 모범생 닉. 12살 때 손을 댄 약물에 어느새 중독자가 되었다.\r\n\r\n“다행히도 아직 이야기는 끝나지 않았어”\r\n아들의 중독이 자기 때문이 아닐까 자책하던 데이비드. 눈물 흘리며 포기하는 대신, 세상의 전부이자 모든 것인 닉의 손을 끝까지 붙잡는데…',
  'comments': [{'comment': '쉽게 손댈 수 있는 것은 손쉽게 잃어버리기 마련.',
    'ratings': '',
    'like': '68',
    'dont_like': '0'},
   {'comment': '티미 안녕 오늘 트레일러가 찔끔 올라와서 기념할까 싶어 행복한 기분으로 여기에 왔어',
    'ratings': '',
    'like': '33',
    'dont_like': '3'},
   {'comment': '과거의 아름다운 기억은 현재의 지옥을 견디기 위한 힘이 되 줄지는 모르지만 이 열차는 이미 지옥행으로 가고 있다. 아버지를 제외한 다른 인물들은 다소 편향적으로 그려지고 음악에 너무 힘을 준 나머지 영상보다 먼저 관객을 데려가는 경우가 가끔 있다. 편집도 솔직히 맘에 안드는 구석이 조금은 있다. 하지만 연기로 시비 걸기 어려운 배우들이니 트집 잡을 거라고는 고작 이정도.  2019 오스카 후보작 시리즈로 당연하다고 생각했었는데. 블랜팬서??? 장난쳐 지금?',
    'ratings': '3.5',
    'like': '18',
    'dont_like': '0'},
   {'comment': '티모시의 약빤연기.. 눈빛 목소리에 치

In [328]:
with open('all_data.txt', 'w', encoding = 'utf-8') as make_file:
    json.dump(all_data, make_file, ensure_ascii=False, indent='\t')

# 스포일러가 있어요 를 누르는 코드. 고생했지만... 필요 없을 듯.
now_html = crawler.page_source
now_source = BeautifulSoup(now_html, 'lxml')
now_source.find_all('button',{'class':'css-1txg5x6-StylelessButton-AcceptSpoilerButton e1xxz10x7'})
num_spoil = len(now_source.find_all('button',{'class':'css-1txg5x6-StylelessButton-AcceptSpoilerButton e1xxz10x7'}))

for i in range(num_spoil):
    crawler.find_elements_by_xpath("//button[@aria-label='Accept Spoiler']")[i].click()